### パラメータの調整
各識別器のパラメータを確認する時は、clf.get_params()で確認することができる。

### ロジスティック回帰

パラメータCに関して
→ロジスティック回帰のcのパラメータを大きくすると正則化が弱く、cの値を小さくすると正則化が強くかかる
→このパラメータのCとは正則化項の学習率みたいなもの。
→C=1/λこのラムダが正則化項をどの程度組み入れるのかを指している。（ラムダが大きければ正則化が強くなり、小さくなると正則化は弱い。）
→https://panda-clip.com/param-c-logreg/
→https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic#sklearn.linear_model.LogisticRegression
→http://ailaby.com/logistic_reg/


### グリットサーチを使用する。

→グリットサーチではデータを訓練データとテストデータに分けてスコアを算出してくれる。
→よってそのスコアを見ながらどこからが過学習になっているかをみることが可能。
→また、以下のようなメソッドを使用することでベストなパラメータを確認することが出来る、
→gs.best_params_ , gs.best_score_ , gs.best_estimator_

★グリットサーチの組み合わせが大きくなってしまった場合は、 RandomizedSearchCVというモジュールを使用すると良い。
→この場合、あくまでもランダムで取得してきた中でベストなスコアを取得するというものなので、実際にはベストなスコアが出てきたとしてもそこからまた深掘りをする必要がある。


### パイプラインを使用する！
パイプラインを使用することによって自動的に主成分分析で次元を圧縮して、そこからモデルを学習させるといった一連の流れを自動的に行うことができる。

### パイプラインとグリットサーチを同時に使用することも可能
また、次元圧縮のパラメータと学習器のパラメータ両方とも同時にグリットサーチにかけることも可能。

In [14]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target

from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=1, 
                  train_size=0.8, 
                  test_size=0.2, 
                  random_state=0)

train_index, test_index = next(ss.split(X, y))

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

# 警告を非表示にする（追加コード）
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

### ロジスティック回帰

パラメータCに関して
→ロジスティック回帰のcのパラメータを大きくすると正則化が弱く、cの値を小さくすると正則化が強くかかる
→このパラメータのCとは正則化項の学習率みたいなもの。
→C=1/λこのラムダが正則化項をどの程度組み入れるのかを指している。（ラムダが大きければ正則化が強くなり、小さくなると正則化は弱い。）
→https://panda-clip.com/param-c-logreg/
→https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic#sklearn.linear_model.LogisticRegression
→http://ailaby.com/logistic_reg/

In [15]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [16]:
clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [17]:
clf.C = 1.0
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9649122807017544

In [18]:
clf.C = 100.0
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9736842105263158

### グリットサーチを使用する。

→グリットサーチではデータを訓練データとテストデータに分けてスコアを算出してくれる。
→よってそのスコアを見ながらどこからが過学習になっているかをみることが可能。
→また、以下のようなメソッドを使用することでベストなパラメータを確認することが出来る、
→gs.best_params_ , gs.best_score_ , gs.best_estimator_

★グリットサーチの組み合わせが大きくなってしまった場合は、 RandomizedSearchCVというモジュールを使用すると良い。
→この場合、あくまでもランダムで取得してきた中でベストなスコアを取得するというものなので、実際にはベストなスコアが出てきたとしてもそこからまた深掘りをする必要がある。


In [19]:
from sklearn.model_selection import GridSearchCV
C_range_exp = np.arange(-15.0,21.0)
C_range = 10 ** C_range_exp

solver_list = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]

param = {'C': C_range , "solver": solver_list}
# n_jobsは並列に計算を走らせる方法
# verboseは表示を行う。
gs = GridSearchCV(clf , param , n_jobs=-1,verbose=2)
gs.fit(X_train , y_train)

/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  wa

GridSearchCV(estimator=LogisticRegression(C=100.0),
             param_grid={'C': array([1.e-15, 1.e-14, 1.e-13, 1.e-12, 1.e-11, 1.e-10, 1.e-09, 1.e-08,
       1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00,
       1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08,
       1.e+09, 1.e+10, 1.e+11, 1.e+12, 1.e+13, 1.e+14, 1.e+15, 1.e+16,
       1.e+17, 1.e+18, 1.e+19, 1.e+20]),
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [21]:
# 全ての結果を確認
gs.cv_results_

{'mean_fit_time': array([0.0128686 , 0.00234804, 0.00077634, 0.0019093 , 0.00163727,
        0.00757594, 0.00158648, 0.00065737, 0.00199199, 0.00156693,
        0.00941167, 0.00186882, 0.00076308, 0.00280704, 0.00178857,
        0.0073667 , 0.00237284, 0.00065575, 0.0017818 , 0.0015758 ,
        0.00802302, 0.00200834, 0.00094085, 0.00190339, 0.00149393,
        0.00552979, 0.00137472, 0.00062056, 0.00179653, 0.00151935,
        0.00580306, 0.00476065, 0.00062699, 0.00206065, 0.00228248,
        0.00544233, 0.00477781, 0.00062528, 0.01520863, 0.00276952,
        0.00419273, 0.00382438, 0.00061703, 0.01370268, 0.0129508 ,
        0.00525284, 0.00439181, 0.00074849, 0.01346335, 0.01391873,
        0.00480194, 0.00394754, 0.00079312, 0.00287457, 0.00838146,
        0.00342412, 0.00377421, 0.00073156, 0.0024827 , 0.0023046 ,
        0.00312743, 0.0026154 , 0.0007422 , 0.00232201, 0.00229774,
        0.00354114, 0.00251989, 0.0008646 , 0.00207262, 0.0025568 ,
        0.00479655, 0.00453339,

In [22]:
gs.best_params_ , gs.best_score_ , gs.best_estimator_

({'C': 10.0, 'solver': 'newton-cg'},
 0.9780219780219781,
 LogisticRegression(C=10.0, solver='newton-cg'))

## サポートベクターマシーン

In [24]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')

C_range_exp = np.arange(-2.0, 5.0)
C_range = 10 ** C_range_exp

param = {'C': C_range,
         'kernel': ['linear', 'rbf']}

gs = GridSearchCV(clf, param, n_jobs=-1, verbose=2, return_train_score=True)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


GridSearchCV(estimator=SVC(gamma='auto'), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
                         'kernel': ['linear', 'rbf']},
             return_train_score=True, verbose=2)

In [25]:
gs.best_params_, gs.best_score_, gs.best_estimator_

({'C': 1.0, 'kernel': 'linear'},
 0.9758241758241759,
 SVC(gamma='auto', kernel='linear'))

### パイプラインを使用する！
パイプラインを使用することによって自動的に主成分分析で次元を圧縮して、そこからモデルを学習させるといった一連の流れを自動的に行うことができる。

In [31]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
estimators = [('pca', PCA(whiten=True)), 
              ('clf', LogisticRegression())]
pipe = Pipeline(estimators)

In [33]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA(whiten=True)), ('clf', LogisticRegression())])

In [35]:
pipe.score(X_test, y_test)

0.9649122807017544

### パイプラインとグリットサーチを同時に使用することも可能
また、次元圧縮のパラメータと学習器のパラメータ両方とも同時にグリットサーチにかけることも可能。

In [36]:
estimators = [('pca', PCA(whiten=True)), 
              ('clf', LogisticRegression(solver='liblinear'))]
pipe = Pipeline(estimators)

param = {'clf__C':[1e-5, 1e-3, 1e-2, 1, 1e2, 1e5, 1e10]} # clf.C

gs = GridSearchCV(pipe, param)
gs.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('pca', PCA(whiten=True)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             param_grid={'clf__C': [1e-05, 0.001, 0.01, 1, 100.0, 100000.0,
                                    10000000000.0]})

In [37]:
gs.best_params_, gs.best_score_, gs.best_estimator_

({'clf__C': 1},
 0.9604395604395604,
 Pipeline(steps=[('pca', PCA(whiten=True)),
                 ('clf', LogisticRegression(C=1, solver='liblinear'))]))

In [38]:
gs.score(X_test, y_test)

0.9649122807017544

In [40]:
# 主成分分析と識別器のパラメータを同時に変更する。
from sklearn.svm import SVC

C_range = [1e-3, 1e-2, 1, 1e2, 1e3]

param = {'clf__C': C_range, 
         'clf__kernel': ['linear', 'rbf'], 
         'pca__whiten': [True, False], 
         'pca__n_components': [30, 20, 10]}

estimators = [('pca', PCA()),
              ('clf', SVC(gamma='auto'))]

pipe = Pipeline(estimators)


from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(pipe, param, n_jobs=-1, verbose=2)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=Pipeline(steps=[('pca', PCA()),
                                             ('clf', SVC(gamma='auto'))]),
                   n_jobs=-1,
                   param_distributions={'clf__C': [0.001, 0.01, 1, 100.0,
                                                   1000.0],
                                        'clf__kernel': ['linear', 'rbf'],
                                        'pca__n_components': [30, 20, 10],
                                        'pca__whiten': [True, False]},
                   verbose=2)

In [41]:
gs.best_params_, gs.best_score_, gs.best_estimator_

({'pca__whiten': False,
  'pca__n_components': 20,
  'clf__kernel': 'linear',
  'clf__C': 1},
 0.9758241758241759,
 Pipeline(steps=[('pca', PCA(n_components=20)),
                 ('clf', SVC(C=1, gamma='auto', kernel='linear'))]))

In [42]:
gs.score(X_test, y_test)

0.9649122807017544

In [43]:
gs.cv_results_

{'mean_fit_time': array([0.00375566, 0.00679331, 0.00384216, 0.00728545, 0.01591625,
        0.00762601, 0.00465226, 0.01756344, 0.0049252 , 0.00260382]),
 'std_fit_time': array([0.00051166, 0.00316384, 0.00167794, 0.00284826, 0.01043838,
        0.00172207, 0.00011208, 0.00439215, 0.00109062, 0.00033272]),
 'mean_score_time': array([0.00083494, 0.00266533, 0.00090232, 0.00074949, 0.00061736,
        0.00065594, 0.00161242, 0.00092368, 0.00126357, 0.0007432 ]),
 'std_score_time': array([0.00011111, 0.00092113, 0.00046604, 0.00015159, 0.00010181,
        0.0002014 , 0.00045893, 0.00028047, 0.00035513, 0.00019783]),
 'param_pca__whiten': masked_array(data=[False, False, False, True, True, False, True, True,
                    True, False],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_pca__n_components': masked_array(data=[20, 30, 20, 30, 30, 10, 30, 10, 20, 30],


[CV] END clf__C=100.0, clf__kernel=rbf, pca__n_components=10, pca__whiten=False; total time=   0.0s
[CV] END clf__C=0.01, clf__kernel=rbf, pca__n_components=10, pca__whiten=False; total time=   0.0s
[CV] END clf__C=1, clf__kernel=linear, pca__n_components=10, pca__whiten=True; total time=   0.0s
[CV] END clf__C=1000.0, clf__kernel=rbf, pca__n_components=10, pca__whiten=False; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=10, pca__whiten=True; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=10, pca__whiten=True; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=30, pca__whiten=True; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=30, pca__whiten=True; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=30, pca__whiten=True; total time=   0.0s
[CV] END clf__C=100.0, clf__kernel=linear, pca__n_components=30, pca__whiten=True; total tim